## MySQL
***


### Instalação

```
sudo apt-get update
sudo apt-get install mysql-server
pip3 install mysql-connector
```

### Entrar no MySQL

```
mysql -u <username> -p
senha: ****
exit
```

Se precisar colocar o host como localhost use o -h

### Backup

```
mysqldump -u <username> -p <database> > backup.sql
```


***
### Conexão
***

In [1]:
from mysql import connector
import getpass

In [2]:
senha = getpass.getpass('Senha para mysql: ')

Senha para mysql: ········


In [3]:
# Connection é o objeto que representa o banco de dados
conexao = connector.connect(
    host="localhost",
    user="root",
    passwd=senha,
    #database="mydb"
)

In [4]:
# Vamos criar um cursor, ou seja, um iterador que permite navegar e manipular os registros do DB.
cursor = conexao.cursor()

***
### Criar banco de dados
***

In [5]:
# Cria um database e use ela
cursor.execute("CREATE DATABASE IF NOT EXISTS mydb")
cursor.execute("USE mydb;")

In [6]:
# Mostrar os banco de dados
cursor.execute("SHOW DATABASES")

In [7]:
for database in cursor:
    print(database)

('information_schema',)
('mydb',)
('mysql',)
('performance_schema',)
('sys',)


***
### Criando uma tabela
***

In [8]:
# Criar uma tabela
cursor.execute("""
CREATE TABLE IF NOT EXISTS cidades (
    id INT AUTO_INCREMENT PRIMARY KEY,
    cidade VARCHAR(255),
    uf VARCHAR(2)
);
""")

In [9]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS clientes (
    id INT AUTO_INCREMENT PRIMARY KEY,
    nome VARCHAR(255) NOT NULL,
    cpf VARCHAR(11) NOT NULL,
    email VARCHAR(100) NOT NULL,
    cidade_id INT,
    criado_em DATE NOT NULL,
    FOREIGN KEY (cidade_id) REFERENCES cidades(id)
);
""")

In [10]:
# Mostrar tabelas
cursor.execute("""SHOW TABLES""")

In [11]:
for table in cursor:
    print(table)

('cidades',)
('clientes',)


***
### Inserindo registros
***

In [12]:
cidades = [('Campinas','SP'),
           ('Sao Paulo','SP'),
           ('Rio de Janeiro','RJ')]

In [13]:
cursor.executemany("""
INSERT INTO cidades (cidade, uf) VALUES (%s, %s)
""", cidades)

In [14]:
clientes = [('Regis', '01119239423', 'regis@gmail.com', 1, '2014-06-08'),
            ('Aloisio', '02320322932', 'aloisio@email.com', 2, '2014-06-09'),
            ('Bruna', '11239428344', 'bruna@email.com', 2, '2014-06-09'),
            ('Matheus', '01293844323', 'matheus@email.com', 3, '2014-06-08')]

In [15]:
cursor.executemany("""
INSERT INTO clientes (nome, cpf, email, cidade_id, criado_em) VALUES (%s, %s, %s, %s, %s)
""", clientes)

In [16]:
# Salvando os dados no BD.
conexao.commit()

In [17]:
# Verifica quantos inserções foram feitas
print(cursor.rowcount, "dados inseridos.")
# Pega o ID do ultimo elemento inseridp
print("ID:", cursor.lastrowid)

4 dados inseridos.
ID: 1


***
### Lendo os dados
***

In [18]:
# Registros de clientes
cursor.execute("""
SELECT * FROM clientes;
""")

In [19]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis@gmail.com', 1, datetime.date(2014, 6, 8))
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9))
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9))
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, datetime.date(2014, 6, 8))


In [20]:
# Registros de cidades
cursor.execute("""
SELECT * FROM cidades;
""")

In [21]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Campinas', 'SP')
(2, 'Sao Paulo', 'SP')
(3, 'Rio de Janeiro', 'RJ')


In [22]:
# Registros de clientes e cidades
cursor.execute("""
SELECT * FROM clientes INNER JOIN cidades ON clientes.cidade_id = cidades.id;
""")

In [23]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis@gmail.com', 1, datetime.date(2014, 6, 8), 1, 'Campinas', 'SP')
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9), 2, 'Sao Paulo', 'SP')
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9), 2, 'Sao Paulo', 'SP')
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, datetime.date(2014, 6, 8), 3, 'Rio de Janeiro', 'RJ')


***
### Alterando os dados
***

In [24]:
id_cliente = 1
novo_email = 'regis_novo@gmail.com'
novo_criado_em = '2014-06-11'

In [25]:
cursor.execute("""
UPDATE clientes
SET email = %s, criado_em = %s
WHERE id = %s
""", (novo_email, novo_criado_em, id_cliente))

In [26]:
conexao.commit()

In [27]:
cursor.execute("""
SELECT * FROM clientes;
""")

In [28]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis_novo@gmail.com', 1, datetime.date(2014, 6, 11))
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9))
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9))
(4, 'Matheus', '01293844323', 'matheus@email.com', 3, datetime.date(2014, 6, 8))


***
### Deletando um registro
***

In [29]:
id_cliente = 4

In [30]:
cursor.execute("""
DELETE FROM clientes
WHERE id = %s
""", (id_cliente,))

In [31]:
conexao.commit()

In [32]:
cursor.execute("""
SELECT * FROM clientes;
""")

In [33]:
for registro in cursor.fetchall():
    print(registro)

(1, 'Regis', '01119239423', 'regis_novo@gmail.com', 1, datetime.date(2014, 6, 11))
(2, 'Aloisio', '02320322932', 'aloisio@email.com', 2, datetime.date(2014, 6, 9))
(3, 'Bruna', '11239428344', 'bruna@email.com', 2, datetime.date(2014, 6, 9))


In [34]:
# Deleta todas as instâncias
cursor.execute("DELETE FROM clientes")
cursor.execute("DELETE FROM cidades")

In [35]:
conexao.commit()

***
### Adicionar uma nova coluna
***

In [36]:
cursor.execute("""
ALTER TABLE clientes
ADD COLUMN bloqueado BOOLEAN;
""")

In [37]:
conexao.commit()

***
### Fechando conexão
***

In [38]:
cursor.execute("DROP TABLE IF EXISTS clientes")
cursor.execute("DROP TABLE IF EXISTS cidades")
cursor.execute("DROP DATABASE IF EXISTS mydb")
conexao.commit()

In [39]:
cursor.close()
conexao.close()